In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_5726: Some(http://gw02.itversity.com:4052)

In [2]:
%showtypes on

Types will be printed.


In [3]:
import org.apache.spark.sql.DataFrame
import util.Random

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.StandardScaler

In [4]:
val dataWithoutHeader = spark
                        .read
                        .option("inferSchema",true)
                        .option("header",false)
                        .csv("hdfs:///user/kranthidr/dataSets/KDDCup1999DataSet")

Waiting for a Spark session to start...

dataWithoutHeader = [_c0: int, _c1: string ... 40 more fields]


[_c0: int, _c1: string ... 40 more fields]

In [5]:
dataWithoutHeader

[_c0: int, _c1: string ... 40 more fields]

In [6]:
dataWithoutHeader.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: d

In [7]:
dataWithoutHeader.take(3)

0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,normal.


In [8]:
val data = dataWithoutHeader.toDF(
        "duration", "protocol_type", "service", "flag",
        "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
        "hot", "num_failed_logins", "logged_in", "num_compromised",
        "root_shell", "su_attempted", "num_root", "num_file_creations",
        "num_shells", "num_access_files", "num_outbound_cmds",
        "is_host_login", "is_guest_login", "count", "srv_count",
        "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
        "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
        "dst_host_count", "dst_host_srv_count",
        "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
        "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
        "dst_host_serror_rate", "dst_host_srv_serror_rate",
        "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
        "label")


data = [duration: int, protocol_type: string ... 40 more fields]


[duration: int, protocol_type: string ... 40 more fields]

In [9]:
data.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [10]:
data.select("label").groupBy("label").count().orderBy($"count".desc).show(25)

+----------------+-------+
|           label|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
|            phf.|      4|
|           perl.|      3|
|            spy.|      2|
+----------------+-------+



In [11]:
val numericOnly = data.drop("protocol_type", "service", "flag").cache()

numericOnly = [duration: int, src_bytes: int ... 37 more fields]


[duration: int, src_bytes: int ... 37 more fields]

In [12]:
val assembler = new VectorAssembler().setInputCols(numericOnly.columns.filter(_!="label")).setOutputCol("featureVector")
val kmeans = new KMeans().setPredictionCol("cluster").setFeaturesCol("featureVector")

assembler = vecAssembler_64f4e128eb04
kmeans = kmeans_5bcea36b73a5


kmeans_5bcea36b73a5

In [13]:
val pipeline = new Pipeline().setStages(Array(assembler, kmeans))

pipeline = pipeline_bd9f9e8bd0f5


pipeline_bd9f9e8bd0f5

In [14]:
val pipelineModel = pipeline.fit(numericOnly)

pipelineModel = pipeline_bd9f9e8bd0f5


pipeline_bd9f9e8bd0f5

In [15]:
val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]

kmeansModel = kmeans_5bcea36b73a5


kmeans_5bcea36b73a5

In [16]:
kmeansModel.clusterCenters.foreach(println)

[48.34019491959669,1834.6215497618625,826.2031900016945,5.7161172049003456E-6,6.487793027561892E-4,7.961734678254053E-6,0.012437658596734055,3.205108575604837E-5,0.14352904910348827,0.00808830584493399,6.818511237273984E-5,3.6746467745787934E-5,0.012934960793560386,0.0011887482315762398,7.430952366370449E-5,0.0010211435092468404,0.0,4.082940860643104E-7,8.351655530445469E-4,334.9735084506668,295.26714620807076,0.17797031701994342,0.17803698940272533,0.05766489875327374,0.05772990937912738,0.7898841322630883,0.021179610609908736,0.028260810096292836,232.98107822302248,189.21428335201279,0.753713389800642,0.030710978823798966,0.6050519309248854,0.006464107887636004,0.17809118431826013,0.17788589813474293,0.05792761150001129,0.05765922142400886]
[10999.0,0.0,1.309937401E9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,255.0,1.0,0.0,0.65,1.0,0.0,0.0,0.0,1.0,1.0]


In [17]:
%lsmagic

Available line magics:
%lsmagic %showtypes %adddeps %truncation %addjar

Available cell magics:
%%sql %%html %%javascript %%dataframe %%pyspark %%scala %%sparkr

Type %<magic_name> for usage info.
         


In [18]:
val withCluster = pipelineModel.transform(numericOnly)

withCluster = [duration: int, src_bytes: int ... 39 more fields]


[duration: int, src_bytes: int ... 39 more fields]

In [19]:
val cluster_label_count = withCluster.select("cluster","label").groupBy("cluster","label").count()

cluster_label_count = [cluster: int, label: string ... 1 more field]


[cluster: int, label: string ... 1 more field]

In [20]:
cluster_label_count.groupBy("cluster").count().show()

+-------+-----+
|cluster|count|
+-------+-----+
|      1|    1|
|      0|   23|
+-------+-----+



In [21]:
cluster_label_count.orderBy($"count".desc,$"cluster").show(25)
cluster_label_count.orderBy($"cluster", $"count").show(25)

+-------+----------------+-------+
|cluster|           label|  count|
+-------+----------------+-------+
|      0|          smurf.|2807886|
|      0|        neptune.|1072017|
|      0|         normal.| 972781|
|      0|          satan.|  15892|
|      0|        ipsweep.|  12481|
|      0|      portsweep.|  10412|
|      0|           nmap.|   2316|
|      0|           back.|   2203|
|      0|    warezclient.|   1020|
|      0|       teardrop.|    979|
|      0|            pod.|    264|
|      0|   guess_passwd.|     53|
|      0|buffer_overflow.|     30|
|      0|           land.|     21|
|      0|    warezmaster.|     20|
|      0|           imap.|     12|
|      0|        rootkit.|     10|
|      0|     loadmodule.|      9|
|      0|      ftp_write.|      8|
|      0|       multihop.|      7|
|      0|            phf.|      4|
|      0|           perl.|      3|
|      0|            spy.|      2|
|      1|      portsweep.|      1|
+-------+----------------+-------+

+-------+----------

In [22]:
def clusterScore0(data: DataFrame, k: Int) = {
    val assembler = new VectorAssembler().setInputCols(data.columns.filter(_!="label")).setOutputCol("featureVector")
    
    val kmeans = new KMeans().setSeed(Random.nextLong()).setK(k).setPredictionCol("cluster").
                setFeaturesCol("featureVector")
    
    val pipeline = new Pipeline().setStages(Array(assembler, kmeans))
    
    val kmeansModel = pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    
    kmeansModel.computeCost(assembler.transform(data))/ data.count()
}

clusterScore0: (data: org.apache.spark.sql.DataFrame, k: Int)Double


In [23]:
val k_kScore = (20 to 280 by 20).map(k => (k, clusterScore0(numericOnly, k))) 

k_kScore = Vector((20,6.982127937698625E7), (40,4.0042114138487235E7), (60,1.257456556431137E7), (80,1.5947382916869406E7), (100,1.1825431477712497E7), (120,1.018226421778534E7), (140,7368383.047505253), (160,9727025.294877974), (180,3368921.756771228), (200,6039924.684517559), (220,8246238.134979978), (240,1667048.2109641863), (260,2778021.1095718606), (280,4254596.4870235175))


Vector((20,6.982127937698625E7), (40,4.0042114138487235E7), (60,1.257456556431137E7), (80,1.5947382916869406E7), (100,1.1825431477712497E7), (120,1.018226421778534E7), (140,7368383.047505253), (160,9727025.294877974), (180,3368921.756771228), (200,6039924.684517559), (220,8246238.134979978), (240,1667048.2109641863), (260,2778021.1095718606), (280,4254596.4870235175))

In [24]:
%AddDeps co.theasi plotly_2.11 0.2.0 --transitive

Marking co.theasi:plotly_2.11:0.2.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir4383593501787824200/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir4383593501787824200/toree_add_deps/https/repo1.maven.org/maven2/org/json4s/json4s-scalap_2.11/3.3.0/json4s-scalap_2.11-3.3.0.jar
-> New file at /tmp/toree-tmp-dir4383593501787824200/toree_add_deps/https/repo1.maven.org/maven2/com/thoughtworks/paranamer/paranamer/2.8/paranamer-2.8.jar
-> New file at /tmp/toree-tmp-dir4383593501787824200/toree_add_deps/https/repo1.maven.org/maven2/org/scalaj/scalaj-http_2.11/2.2.1/scalaj-http_2.11-2.2.1.jar
-> New file at /tmp/toree-tmp-dir4383593501787824200/toree_add_deps/https/repo1.maven.org/maven2/com/thoughtworks/paranamer/paranamer/2.8/paranamer-2.8.pom
-> New file at /tmp/toree-tmp-dir4383593501787824200/toree_add_deps/https/repo1.maven.org/maven2/org/scalaj/scalaj-http_2.11/2.2.1/scalaj-http_2.11-2.2.1.pom
-> New file at /tmp/toree-tmp-dir4383593

In [25]:
 k_kScore.map(_._1)

Vector(20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280)

In [26]:
 k_kScore.map(_._2)

Vector(6.982127937698625E7, 4.0042114138487235E7, 1.257456556431137E7, 1.5947382916869406E7, 1.1825431477712497E7, 1.018226421778534E7, 7368383.047505253, 9727025.294877974, 3368921.756771228, 6039924.684517559, 8246238.134979978, 1667048.2109641863, 2778021.1095718606, 4254596.4870235175)

In [27]:
import co.theasi.plotly._

In [28]:
val xs =  k_kScore.map(_._1)
val ys =  k_kScore.map(_._2)

val plot = Plot().withScatter(xs, ys)

draw(plot, "clustering_elbow_ch05_traffic_anamoly")

xs = Vector(20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280)
ys = Vector(6.982127937698625E7, 4.0042114138487235E7, 1.257456556431137E7, 1.5947382916869406E7, 1.1825431477712497E7, 1.018226421778534E7, 7368383.047505253, 9727025.294877974, 3368921.756771228, 6039924.684517559, 8246238.134979978, 1667048.2109641863, 2778021.1095718606, 4254596.4870235175)
plot = CartesianPlot(Vector(Scatter(Vector(PInt(20), PInt(40), PInt(60), PInt(80), PInt(100), PInt(120), PInt(140), PInt(160), PInt(180), PInt(200), PInt(220), PInt(240), PInt(260), PInt(280)),Vector(PDouble(6.982127937698625E7), PDouble(4.0042114138487235E7), PDouble(1.257456556431137E7), PDouble(1...


CartesianPlot(Vector(Scatter(Vector(PInt(20), PInt(40), PInt(60), PInt(80), PInt(100), PInt(120), PInt(140), PInt(160), PInt(180), PInt(200), PInt(220), PInt(240), PInt(260), PInt(280)),Vector(PDouble(6.982127937698625E7), PDouble(4.0042114138487235E7), PDouble(1.257456556431137E7), PDouble(1.5947382916869406E7), PDouble(1.1825431477712497E7), PDouble(1.018226421778534E7), PDouble(7368383.047505253), PDouble(9727025.294877974), PDouble(3368921.756771228), PDouble(6039924.684517559), PDouble(8246238.134979978), PDouble(1667048.2109641863), PDouble(2778021.1095718606), PDouble(4254596.4870235175)),ScatterOptions(None,List(),None,MarkerOptions(None,None,None,None,None)))),CartesianPlotOptions(Axis(AxisOptions(None,None,None,None,None,None,None,Font(None,None,None),Font(None,None,None),None,None,None,None)),Axis(AxisOptions(None,None,None,None,None,None,None,Font(None,None,None),Font(None,None,None),None,None,None,None))))